<table>
<tr>
<td width=15%><img src="../../img/UGA.png"></img></td>
<td><center><h1>Project n°3</h1></center></td>
<td width=15%><a href="https://team.inria.fr/tripop/team-members/" style="font-size: 16px; font-weight: bold">Florian Vincent</a> </td>
</tr>
</table>

In [ ]:
# Importation des packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support as score

import pickle as pk

In [2]:
%load_ext autoreload
%autoreload 2

# Learning text classification

This project is heavily inspired from [Jigsaw's *Toxic Comments Classification* challenge](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/overview) on kaggle.
To avoid copy-pastings of foreign code, it will guide you towards specific tools to test and use.

## Overview of the project

Take a look at the *zip*ed csv data files by unzipping them (`for name in $(ls *.zip); do unzip $name; done;`).

Every comment in the train set is classified with a label in `{"toxic", "severe_toxic", "obscene", "threat", "insult", "identity hate"}`.
You will need to train multiple kind of models to identify those comments, and you will test them against the test dataset.

# ***Début du projet***

Dans cette section, nous chargeons les fichiers de données CSV contenant les commentaires et leurs étiquettes associées. Ces données sont divisées en trois parties :
- **Entraînement** : Les commentaires avec leurs labels pour entraîner le modèle.
- **Test** : Les commentaires sans labels pour l'évaluation.
- **Labels de test** : Les étiquettes correspondantes aux données test, utilisées pour évaluer la performance du modèle.

In [ ]:
# Chargement des données
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")
print(train.head())

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# Recherche des valeurs manquantes
train.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

# ***Nettoyage des données***

In [6]:
# Suppression de la ponctuation et des sauts de lignes
train["comment_clean"] = train["comment_text"].apply(lambda x : re.sub("[^a-zA-Z]", ' ', x))

In [8]:
# Conversion en minuscule
train["comment_clean"] = train["comment_clean"].str.lower()

In [ ]:
# Visualisation des données d'entraînement après les modifications précédentes
train[["comment_text", "comment_clean"]].head(6)

,comment_text,comment_clean
0,Explanation\nWhy the edits made under my usern...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,d aww he matches this background colour i m s...
2,"Hey man, I'm really not trying to edit war. It...",hey man i m really not trying to edit war it...
3,"""\nMore\nI can't make any real suggestions on ...",more i can t make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",you sir are my hero any chance you remember...
5,"""\n\nCongratulations from me as well, use the ...",congratulations from me as well use the to...


In [10]:
# Tokenisation (séparation mot à mot)
train["comment_clean"] = train["comment_clean"].apply(word_tokenize)

In [ ]:
# Visualisation des données d'entraînement après les modifications précédentes
train[["comment_text", "comment_clean"]].head(6)

,comment_text,comment_clean
0,Explanation\nWhy the edits made under my usern...,"[explanation, why, the, edits, made, under, my..."
1,D'aww! He matches this background colour I'm s...,"[d, aww, he, matches, this, background, colour..."
2,"Hey man, I'm really not trying to edit war. It...","[hey, man, i, m, really, not, trying, to, edit..."
3,"""\nMore\nI can't make any real suggestions on ...","[more, i, can, t, make, any, real, suggestions..."
4,"You, sir, are my hero. Any chance you remember...","[you, sir, are, my, hero, any, chance, you, re..."
5,"""\n\nCongratulations from me as well, use the ...","[congratulations, from, me, as, well, use, the..."


In [12]:
# Suppression des stopwords (mots de "liaisons" inutiles)
stop_words = set(stopwords.words("english"))

train["comment_clean"] = train["comment_clean"].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
# Visualisation des données d'entraînement après les modifications précédentes
train[["comment_text", "comment_clean"]].head(6)

,comment_text,comment_clean
0,Explanation\nWhy the edits made under my usern...,"[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,"[aww, matches, background, colour, seemingly, ..."
2,"Hey man, I'm really not trying to edit war. It...","[hey, man, really, trying, edit, war, guy, con..."
3,"""\nMore\nI can't make any real suggestions on ...","[make, real, suggestions, improvement, wondere..."
4,"You, sir, are my hero. Any chance you remember...","[sir, hero, chance, remember, page]"
5,"""\n\nCongratulations from me as well, use the ...","[congratulations, well, use, tools, well, talk]"


In [14]:
# Lemmatisation (suppression des pluriels, conjugaisons, etc.)
lemmatizer = WordNetLemmatizer()

train["comment_clean"] = train["comment_clean"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
# Visualisation des données d'entraînement après les modifications précédentes
train[["comment_text", "comment_clean"]].head(6)

,comment_text,comment_clean
0,Explanation\nWhy the edits made under my usern...,"[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,"[aww, match, background, colour, seemingly, st..."
2,"Hey man, I'm really not trying to edit war. It...","[hey, man, really, trying, edit, war, guy, con..."
3,"""\nMore\nI can't make any real suggestions on ...","[make, real, suggestion, improvement, wondered..."
4,"You, sir, are my hero. Any chance you remember...","[sir, hero, chance, remember, page]"
5,"""\n\nCongratulations from me as well, use the ...","[congratulation, well, use, tool, well, talk]"


In [ ]:
# Reconversion des listes en chaînes de charactères
train["comment_clean"] = train["comment_clean"].apply(lambda x: " ".join(x))

In [17]:
# Visualisation des données d'entraînement après les modifications précédentes
train[["comment_text", "comment_clean"]].head(6)

,comment_text,comment_clean
0,Explanation\nWhy the edits made under my usern...,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",hey man really trying edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",make real suggestion improvement wondered sect...
4,"You, sir, are my hero. Any chance you remember...",sir hero chance remember page
5,"""\n\nCongratulations from me as well, use the ...",congratulation well use tool well talk


In [ ]:
# Regroupement de toutes les modifications précédentes dans une fonction
# Afin de pouvoir les effectuer plus rapidement sur tous les fichiers (train et test)
def nettoyage(df) :
    # Suppression de la ponctuation et des sauts de lignes
    df["comment_clean"] = df["comment_text"].apply(lambda x : re.sub("[^a-zA-Z]", ' ', x))

    # Conversion en minuscule
    df["comment_clean"] = df["comment_clean"].str.lower()

    # Tokenisation
    df["comment_clean"] = df["comment_clean"].apply(word_tokenize)

    # Suppression des stopwords
    stop_words = set(stopwords.words("english"))
    df["comment_clean"] = df["comment_clean"].apply(lambda x: [word for word in x if word not in stop_words])

    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    df["comment_clean"] = df["comment_clean"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

    # Reconversion des listes en chaînes de charactères
    df["comment_clean"] = df["comment_clean"].apply(lambda x: " ".join(x))

    return df

# Attention : la colonne de texte à traiter doit impérativement s'appeler "comment_text".

# ***Vectorisation***

In [1]:
documents = train["comment_clean"]
vectorizer = CountVectorizer(max_features=10000)
# Adaptation et transformation des documents en une matrice comptant le nombre d'occurences des mots
X = vectorizer.fit_transform(documents)
print("Shape of sparse matrix:", X.shape)  # Dimensions : nombre de documents × nombre de mots uniques
print("Vocabulary:\n", vectorizer.get_feature_names_out())
print(X.toarray())

NameError: name 'train' is not defined

La méthode TF-IDF attribue un poids à chaque mot en fonction de sa fréquence dans un document et de son importance. Cela permet de réduire l'impact des mots très fréquents comme "le", "est",...

In [2]:
# Chargement des documents
documents = train["comment_clean"]

# Initialisisation TfidfVectorizer avec réduction de vocabulaire
tfidf_vectorizer = TfidfVectorizer(max_features=10000, min_df=2, max_df=0.9)
    # min_df = 2 signifie que seuls les mots apparaissant dans au moins 2 documents seront inclus.
    # max_df = 0.9 signifie que les mots présents dans plus de 90% des documents seront ignorés.

# Transformation en matrice TF-IDF
X_tfidf = tfidf_vectorizer.fit_transform(documents)

# Résumé de la matrice
print("Shape of sparse matrix (TF-IDF):", X_tfidf.shape)
print("Feature names (vocabulary example):", tfidf_vectorizer.get_feature_names_out()[:10])

# Choix du 3ème commentaire pour inspection
first_comment_tfidf = X_tfidf[2].toarray()[0]

# Obtention du vocabulaire (features)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Création d'une liste des mots et de leurs poids TF-IDF (non nuls uniquement)
non_zero_indices = first_comment_tfidf.nonzero()[0]
words_weights = [(feature_names[i], first_comment_tfidf[i]) for i in non_zero_indices]

# Trie des mots par ordre décroissant de poids
words_weights_sorted = sorted(words_weights, key=lambda x: x[1], reverse=True)
tfidf_vectorizer = TfidfVectorizer(max_features=10000, min_df=2, max_df=0.9)

# Affichage des résultats
print("\nWords and their TF-IDF weights for the first comment:")
for word, weight in words_weights_sorted:
    print(f"{word}: {weight}")

NameError: name 'train' is not defined

# Réduction de dimensions
**Find a suitable dimension reduction technique to study the structure of the data. Display your findings with visual means (you can use `seaborn`).**
L'ACP classique, telle qu'implémentée dans sklearn.decomposition.PCA, nécessite que la matrice de données soit dense. Cela implique que, pour les matrices sparse (creuses, contenant majoritairement des zéros), une conversion en matrice dense est nécessaire avant de pouvoir effectuer l'ACP, ce qui peut entraîner une consommation excessive de mémoire.

En revanche, TruncatedSVD est spécifiquement conçu pour traiter directement les matrices sparse, telles que celles générées par les représentations TF-IDF en NLP. Cela permet d'économiser une quantité significative d'espace mémoire en évitant toute conversion inutile.

Par ailleurs, l'ACP classique repose sur une décomposition complète des matrices, via une méthode connue sous le nom de décomposition en valeurs propres (eigendecomposition). Cette approche est particulièrement coûteuse en temps pour des matrices de grande taille et consomme une quantité importante de mémoire.

À l'inverse, TruncatedSVD utilise une technique appelée décomposition SVD tronquée, qui calcule uniquement les premiers n composants principaux sans générer l'intégralité de la matrice de covariance. Cela rend TruncatedSVD non seulement plus rapide, mais également bien plus efficace pour manipuler des matrices de grande dimension, tout en réduisant le coût computationnel.

In [ ]:
# Réduction de dimensions avec TruncatedSVD

svd = TruncatedSVD(n_components = 2)  # Réduction à 2 dimensions pour visualisation
X_svd = svd.fit_transform(X_tfidf)  # Pas besoin de convertir en dense

# Chargement des étiquettes 
labels = train["toxic"]

# Création d'un DataFrame pour combiner les composantes SVD et les étiquettes
svd_df = pd.DataFrame(data = X_svd, columns = ["SVD1", "SVD2"])
svd_df["Label"] = labels
# Visualisation
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x="SVD1", y="SVD2", hue="Label", data=svd_df, palette="viridis", alpha=0.6
)
plt.title("TruncatedSVD Visualization of Text Data")
plt.xlabel("SVD Component 1")
plt.ylabel("SVD Component 2")
plt.legend(title="Label")
plt.show()

La majorité des points (documents) se situent dans une région dense autour des coordonnées proches de (0.2, 0.2).
Les données sont bien projetées sur deux dimensions, ce qui permet une interprétation visuelle.
Les points en bleu (label 0) sont largement dominants dans les données, ce qui indique un déséquilibre des classes (beaucoup plus de textes non toxiques que toxiques).

Il est donc nécessaire de vérifier le ratio entre les classes 0 et 1 pour confirmer le déséquilibre. \
Ce déséquilibre pour toxic n'est pas dû à SVD, mais aux proportions intrinsèques des données. SVD ne favorise pas une classe ou une autre ; il se concentre uniquement sur la variance des données. \
SVD gère efficacement les matrices creuses de TF-IDF et conserve des tendances globales dans les données, ce qui est utile pour une analyse exploratoire.

In [ ]:
print(train["toxic"].value_counts(normalize=True))
X_tfidf.shape

### Combien de dimensions sont suffisantes ?

Une bonne pratique consiste à examiner la variance expliquée cumulée pour choisir un nombre optimal de dimensions. En analysant la courbe cumulative de la variance expliquée (avec cumulative_variance), nous constatons que les 300 premières dimensions couvrent généralement 35% de la variance totale dans des tâches de ce projet NLP. Cela signifie que la majorité des informations pertinentes des données d'origine est maintenue. En choisissant 300 dimensions, couvrant déjà la majorité de la variance expliquée, des valeurs plus élevées risquent d'augmenter le coût computationnel sans apporter de gain significatif.

In [ ]:
# Ajustement SVD avec plus de composantes pour analyser la variance expliquée
svd = TruncatedSVD(n_components=1000, random_state=42)
svd.fit(X_tfidf)

# Variance expliquée cumulative
cumulative_variance = svd.explained_variance_ratio_.cumsum()

# Visualisation
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance)
plt.axhline(y=0.35, color='r', linestyle='--')  # Ligne pour 35% de variance expliquée
plt.xlabel("Nombre de dimensions")
plt.ylabel("Variance expliquée cumulée")
plt.title("Analyse de la Variance Expliquée par TruncatedSVD")
plt.grid()
plt.show()

D'après la figure ci dessous, nous pouvons observer que les premières composantes expliquent la majeure partie de la variance. La variance expliquée diminue rapidement après les premières composantes (effet d'"écrasement"). Cela indique qu'on peut probablement réduire de manière significative le nombre de dimensions sans perdre beaucoup d'information. Ce qui accélérera nos calculs pour les modèles comme la SVM par exemple.

In [ ]:
# Ajustement SVD avec plus de composantes pour analyser la variance expliquée
svd = TruncatedSVD(n_components=1000, random_state=42)
svd.fit(X_tfidf)

# Variance expliquée par chaque composante
explained_variance = svd.explained_variance_ratio_

# Visualisation de l'histogramme avec les 300 premières dimensions en rouge
plt.figure(figsize=(12, 6))
colors = ['red' if i < 300 else 'blue' for i in range(len(explained_variance))]
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color=colors, edgecolor='black')
plt.axvline(x=300, color='green', linestyle='--', label="300 Dimensions (Ligne de coupe)")  # Ligne pour 300 dimensions
plt.xlabel("Numéro de la composante")
plt.ylabel("Variance expliquée")
plt.title("Histogramme de la Variance Expliquée par TruncatedSVD")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# ***Classifications***

### ***Préparation des données communes***

In [ ]:
# Chargement des fichiers CSV
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")

In [ ]:
# Nettoyage des données
train = nettoyage(train)
test = nettoyage(test)

Les étiquettes des données de test contiennent des valeurs `-1`, qui indiquent des commentaires sans étiquettes valides.  
Nous les supprimons donc des jeux de données.

In [3]:
test = test[test_labels["toxic"] != -1]
test_labels = test_labels[test_labels["toxic"] != -1]

NameError: name 'test' is not defined

### ***Régression Logistique***

La régression logistique est un modèle simple mais efficace pour une classification binaire.  
Nous entraînons un modèle pour prédire si un commentaire est "toxique" (`1`) ou "non-toxique" (`0`).

In [ ]:
## with the use of tf-idf the words thats are more accurate are less important so we wont have the redondant words when it comes to counting from sklearn.feature_extraction.text import TfidfVectorizer
# The TfidfVectorizer: tokenazation  splits the text into smaller units called tokens (usually words)
# then vectorizer calculates the TF-IDF (Term Frequency-Inverse Document Frequency) value for each term in the corpus.
# which is the number of a token on the total number of all tokens  for term freq 
# Inverse Document Frequency or idf : is simply the log of the number of  documents or sentences thats containes the token (word)

# Pipeline avec l'étape de TF-IDF
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000))  # Étape TF-IDF
])

# Chargement du corpus
corpus1 = train["comment_clean"]

# Application du pipeline
X_tfidf = pipeline.fit_transform(corpus1)

# Affichage des dimensions et des vecteurs train 
print("Shape de X_train :", X_tfidf.shape)  # nombre de documents x nombre de tokens (words)

In [ ]:
test = test.merge(test_labels, on = "id")
# Convertir les colonnes à partir de la 3ème colonne en numériques
# test.iloc[:, 3:] = test.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')
# Filtrage des lignes où la somme des colonnes à partir de la 3ème est >= 0
test = test[test.iloc[:, 3:].sum(axis=1) >= 0]
test.head()

**Paramètres choisis :**
- `solver='liblinear'` : Adapté aux  problèmes de classification binaire.
- `C=1.0` : Paramètre de régularisation (valeur par défaut).
- `class_weight='balanced'` : Pour compenser le déséquilibre des classes.

Nous évaluerons le modèle avec un rapport de classification et une matrice de confusion.

In [ ]:
## logistic classifier 
# les commentaires toxiques.
model = LogisticRegression(solver='liblinear', C=0.1,class_weight='balanced', random_state=0)

# X : Les commentaires transformés en vecteurs numériques avec TF-IDF
# X_tfidf est la matrice résultante de la transformation des commentaires nettoyés en une représentation numérique 
# (chaque ligne est un commentaire, et chaque colonne représente un mot, avec les valeurs TF-IDF associées).
X_train = X_tfidf
# Y : Le label (binaire pour toxicité)
# Ici, nous utilisons la colonne 'toxic' de 'train' comme étiquette cible.
# Cette colonne contient des valeurs 0 ou 1, représentant si le commentaire est toxique ou non.
y_train = train["toxic"]
# Entraînement du modèle avec les données X_train (les vecteurs des commentaires)
# et y_train (les étiquettes correspondantes de toxicité).
model.fit(X_train, y_train)
# Après l'entraînement du modèle, on peut maintenant faire des prédictions sur de nouvelles données.
# Affiche les classes que le modèle peut prédire.
# Cela montre les étiquettes de classification possibles (dans ce cas, 'toxic' ou 'non-toxic' pour un modèle binaire).
# présenter par la liste 0,1
print(model.classes_)
# Affiche l'ordonnée à l'origine du modèle (le biais).
print(model.intercept_)
# Affiche les coefficients du modèle pour chaque caractéristique (mot dans ce cas).
print(model.coef_)

In [ ]:
# Sauvegarde du modèle et du vectoriseur avec pickle
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

#### Le modèle régression logistique avec ACP

Les résultats sont similaires avec ou sans ACP, dans ce cas présent. Nous avons donc sélectionné celui sans ACP.

In [ ]:
# # avec pca 
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, confusion_matrix

# # Étape 1 : Réduction de la dimensionnalité avec PCA
# pca = PCA(n_components=300, random_state=0)
# X_train_reduced = pca.fit_transform(X_train)

# # Étape 2 : Régression logistique avec les données réduites
# model2 = LogisticRegression(solver='liblinear', C=0.1, class_weight='balanced', random_state=0)

# # Entraînement du modèle
# model2.fit(X_train_reduced, y_train)

# # Étape 3 : Résultats du modèle
# print("Classes prédites :", model2.classes_)
# print("Ordonnée à l'origine (biais) :", model2.intercept_)
# print("Coefficients du modèle :", model2.coef_)

#### Évaluation sur les données de test

Nous utilisons le modèle pour prédire les étiquettes des données de test et évaluons sa performance avec :
1. Un **rapport de classification** (précision, rappel, F1-score).
2. Une **matrice de confusion** pour visualiser les résultats.

Cela nous donne des insights sur la capacité du modèle à généraliser sur les autre labels .

In [ ]:
# Prédictions sur les données de test
# Transformation des données de test avec le même vectoriseur (sans fit)
X_test_tfidf_t = pipeline.transform(test["comment_clean"])  
y_test = test["toxic"]  # Remplace par les vraies étiquettes de test

# Prédictions
y_pred = model.predict(X_test_tfidf_t)

# Affichage du rapport de classification
print("Classification Report:\n", classification_report(y_test, y_pred))

# Affichage de la matrice de confusion
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0', 'Predicted 1'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0', 'Actual 1'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()

#### Classification multi-étiquettes avec Régression Logistique

Pour traiter plusieurs étiquettes (`toxic`, `severe_toxic`, etc.), nous utilisons un classifieur multi-sortie (`MultiOutputClassifier`).  
Chaque étiquette est prédite individuellement avec un modèle de régression logistique.
Voici les choix des paramètres pour ce modèle :

1. **`solver='liblinear'`** :  
   Ce solveur est bien adapté aux problèmes de classification binaire. Chaque étiquette étant une classification binaire dans notre cas, ce solveur est optimal.

2. **`C=1.0`** :  
   Ce paramètre contrôle la régularisation. Une valeur par défaut de `1.0` équilibre la régularisation et l’ajustement du modèle. Cela permet de prévenir le surapprentissage sans sous-apprentissage.

3. **`class_weight='balanced'`** :  
   Les classes dans les données sont déséquilibrées (ex. peu de commentaires `toxic` par rapport aux `non-toxic`). Ce paramètre ajuste automatiquement les poids des classes en fonction de leur fréquence dans les données d'entraînement, ce qui améliore les performances pour les classes rares.

4. **`random_state=0`** :  
   Permet de fixer une graine pour la reproductibilité des résultats.
   
Les résultats pour chaque étiquette sont évalués à l'aide des métriques suivantes :
- Précision : proportion des prédictions positives (1) correctes parmi toutes les prédictions positives. Elle évalue combien des prédictions positives sont réellement correctes.
- Rappel : proportion des exemples positifs correctement identifiés parmi tous les exemples positifs. Il mesure la capacité du modèle à trouver tous les cas positifs.
- F1-score : mesure qui combine la précision et le rappel en une seule valeur. Elle donne une moyenne harmonique des deux et est particulièrement utile lorsque les classes sont déséquilibrées.
- Exactitude (accuracy) : proportion des prédictions correctes, positives ou négatives, par rapport à l'ensemble des prédictions totales. Elle évalue la performance globale du modèle. 

In [ ]:
# Liste des étiquettes
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Données cibles multi-étiquettes (binaire pour chaque étiquette)
y_train_m = train[labels]
X_train_m = X_tfidf
# Modèle multi-étiquette
multi_target_model = MultiOutputClassifier(LogisticRegression(solver='liblinear', C=1.0,class_weight='balanced', random_state=0))

# Entraînement du modèle
multi_target_model.fit(X_train_m, y_train_m)

# Prédictions
X_test_tfidf_t = pipeline.transform(test["comment_clean"])
y_pred_multi = multi_target_model.predict(X_test_tfidf_t)

# Initialisisation d'un DataFrame pour stocker les résultats
results = []

# Évaluation de chaque étiquette
for i, label in enumerate(labels):
    print(f"Évaluation de l'étiquette: {label}")
    
    # Obtention des vraies étiquettes et les prédictions pour chaque étiquette
    y_test_label = test[label]
    y_pred_label = y_pred_multi[:, i]
    
    # Calcul du rapport de classification
    report = classification_report(
        y_test_label, 
        y_pred_label, 
        zero_division=0,  # Évite les avertissements
        output_dict=True
    )
    
    # Calcul de l'accuracy pour chaque étiquette
    accuracy = accuracy_score(y_test_label, y_pred_label)
    
    # Ajout des résultats au tableau
    results.append({
        "Label": label,
        "Precision": report['weighted avg']['precision'],
        "Recall": report['weighted avg']['recall'],
        "F1-Score": report['weighted avg']['f1-score'],
        "Accuracy": accuracy  # Ajouter l'accuracy à chaque étiquette
    })

# Affichage des résultats sous forme de tableau
results_df = pd.DataFrame(results)
print(results_df)
print("\n \n Précision générale : ", accuracy_score(y_test_label,y_pred_label))

#### Résultats finaux

Le modèle multi-étiquette a permis de prédire les différentes catégories de toxicité.  
Les performances (précision, rappel, F1-score) varient selon les étiquettes, mais montrent que la régression logistique est efficace pour ce problème.  
Des modèles plus avancés pourraient améliorer ces résultats, notamment pour les classes déséquilibrées.
Il existe également un score de permutation permettant d'observer l'importance des variables, ce qui pourrait améliorer le modèle. 
De plus, cette méthode donne des résultats probabilistes pour chaque individu d'appartenir à une classe, ce qui permet d'effectuer ensuite des tests et donc d'observer également les p-valeurs. 
Cette méthode peut apporter également d'autres indicateurs tels que les odds-ratio, qui apportent d'autres informations et d'autres interprétations sur les résultats du modèle.

#### Les classes predites 

In [5]:
# Maintenant, pour chaque commentaire, on print l'étiquette qui a la plus haute probabilité .
# for i, pred in enumerate(y_pred_multi):
    # pred_class = labels[pred.argmax()]                   # Trouver l'indice de la classe avec la plus haute probabilité
    # print(f"Commentaire {i+1} : {pred_class}")

### ***SVM***

Le sous-échantillonnage a été choisi pour travailler avec un volume de données gérable tout en préservant la diversité, ce qui améliore la vitesse d'exécution. La combinaison TF-IDF et SVD a été utilisée pour convertir efficacement les textes en vecteurs numériques et réduire leur dimensionnalité, préparant ainsi les données pour un traitement optimal par le SVM.

#### Tester que sur 15 000 observations

In [ ]:
# Réduction du nombre d'observations 
train = train.sample(n=15000, random_state=42)
test = test.sample(n=10000, random_state=42)

# Initialisation du TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf_train = tfidf_vectorizer.fit_transform(train["comment_clean"])
X_tfidf_test = tfidf_vectorizer.transform(test["comment_clean"])

# Réduction de dimension avec TruncatedSVD
svd = TruncatedSVD(n_components=300, random_state=42)
X_tfidf_train_reduced = svd.fit_transform(X_tfidf_train)
X_tfidf_test_reduced = svd.transform(X_tfidf_test)

Pour tester différents paramètres de C, kernel et gamma et choisir les meilleurs. Nous pouvons utiliser une validation croisée avec un grid search pour explorer plusieurs combinaisons de ces paramètres. Le GridSearchCV de scikit-learn est parfait pour ce genre d'expérimentation.

Après une exécution prolongée de 43 minutes, les résultats obtenus pour la catégorie "toxic" par exemple sont prometteurs. La recherche des hyperparamètres a révélé une configuration optimale avec un noyau RBF, une valeur de C à 0.1 et un gamma à 0.1. Cette configuration a permis d'atteindre une précision remarquable de 0.9395 sur l'ensemble de validation, et une accuracy de 0.927 sur l'ensemble de test

In [ ]:
# Catégorie cible
category = "toxic"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Définir la grille de paramètres à tester
param_grid = {
    'C': [0.1, 1.0, 10.0],  # Valeurs de régularisation
    'kernel': ['linear', 'rbf'],  # Noyaux à tester
    'gamma': ['scale', 'auto', 0.1, 1.0]  # Paramètre gamma pour le noyau RBF
}

# Initialisation du SVM
svm_clf = SVC(class_weight="balanced", random_state=42)

# Initialisation du GridSearchCV avec validation croisée (par exemple, 5 folds)
grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entraîner le modèle sur les données d'entraînement
grid_search.fit(X_tfidf_train_reduced, y_train_category)

# Afficher les meilleurs paramètres et la meilleure précision
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure précision obtenue : ", grid_search.best_score_)

# Utiliser le meilleur modèle pour faire des prédictions sur le test
best_svm_clf = grid_search.best_estimator_
y_pred_category = best_svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy sur le test :", accuracy_score(y_test_category, y_pred_category))
print("Classification Report sur le test :\n", classification_report(y_test_category, y_pred_category))

#### Sans validation croisée pour la catégorie "toxic" :

In [ ]:
# Catégorie cible
category = "toxic"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Initialisation et entraînement du modèle SVM: 
svm_clf = SVC(kernel="rbf", C=0.1,gamma=0.1, class_weight="balanced",random_state=42)
svm_clf.fit(X_tfidf_train_reduced, y_train_category)

# Prédiction sur le jeu de test
y_pred_category = svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy:", accuracy_score(y_test_category, y_pred_category))
print("Classification Report:\n", classification_report(y_test_category, y_pred_category))

#### Avec validation croisée pour la catégorie "toxic" :

In [ ]:
# Catégorie cible
category = "toxic"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

svm_clf = SVC(kernel="rbf", C=0.1, gamma=0.1, class_weight="balanced",random_state=42)

# Effectuer la validation croisée  avec 5 plis
cv_results = cross_validate(svm_clf, X_tfidf_train_reduced, y_train_category, cv=5, scoring='accuracy', return_train_score=False)

# Afficher les résultats de la validation croisée
print("Précision moyenne de la validation croisée : ", cv_results['test_score'].mean())
print("Scores de précision pour chaque pli : ", cv_results['test_score'])

# Initialisation et entraînement final du modèle SVM avec les données d'entraînement
svm_clf.fit(X_tfidf_train_reduced, y_train_category)

# Prédiction sur le jeu de test
y_pred_category = svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy sur le test :", accuracy_score(y_test_category, y_pred_category))
print("Classification Report sur le test :\n", classification_report(y_test_category, y_pred_category))

En comparant les résultats avec et sans validation croisée, on constate une précision globale (Accuracy) similaire d'environ 0.904, indiquant que la validation croisée n'a pas engendré de surapprentissage ou sous-apprentissage significatif. Cependant, la classification reste fortement déséquilibrée : la classe majoritaire (0) est très bien prédite (précision 0.90, recall 1.00), tandis que la classe minoritaire (1) est mal détectée (précision 0.96, mais recall seulement 0.02, F1-Score 0.04). Bien que la validation croisée n'améliore pas la performance finale, elle démontre la cohérence du modèle à travers les différents plis.

#### SVM avec toutes les catégories :

Les performances du modèle varient considérablement selon les catégories. Pour "toxic" et "obscene", les F1-scores de 0.58 et 0.63 respectivement pour la classe 1 sont acceptables, montrant une certaine efficacité dans la détection des classes minoritaires. La catégorie "insult" montre également des résultats prometteurs avec un F1-score de 0.56 pour la classe 1, une précision de 0.59 et un rappel de 0.53, ce qui est relativement bon compte tenu du déséquilibre des classes (580 exemples positifs contre 9420 négatifs). Il est important de noter que les accuracy pour toxic, obscene et insult sont respectivement de 0.92, 0.96 et 0.95, ce qui indique une bonne performance globale du modèle pour ces catégories.Cependant, pour "severe_toxic", "threat", et "identity_hate", les performances sont très faibles, avec des précisions et F1-scores extrêmement bas pour les classes 1, malgré parfois un rappel (recall) élevé. Ce problème est principalement dû au déséquilibre important des données, avec moins de 1% d'exemples positifs dans ces catégories, ce qui biaise le modèle SVM vers la classe majoritaire (0) et rend la détection des classes positives très difficile.

Enfin, il est à noter que le temps d'exécution pour ce code est d'environ 10 minutes.

In [ ]:
# Liste des catégories à tester (par exemple, "toxic", "severe_toxic", "obscene", ...)
categories = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Résultats pour chaque catégorie
for category in categories:
    print(f"=== Catégorie : {category} ===")
    
    # Labels pour la catégorie cible
    y_train_category = train[category]
    y_test_category = test[category]
    
    # Initialisation et entraînement du modèle SVM
    svm_clf = SVC(kernel="rbf", C=0.1, gamma=0.1, class_weight="balanced",random_state=42)
    svm_clf.fit(X_tfidf_train_reduced, y_train_category)
    
    # Prédiction sur le jeu de test
    y_pred_category = svm_clf.predict(X_tfidf_test_reduced)
    
    # Évaluation des performances
    print("Accuracy:", accuracy_score(y_test_category, y_pred_category))
    print("Classification Report:\n", classification_report(y_test_category, y_pred_category))

#### Trouver les bons parametres de SVM pour les catégories severe_toxic, threat et identity_hate

In [ ]:
# Catégorie cible
category = "threat"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Définir la grille de paramètres à tester
param_grid = {
    'C': [0.1, 1.0, 10.0],  # Valeurs de régularisation
    'kernel': ['linear', 'rbf'],  # Noyaux à tester
    'gamma': ['scale', 'auto', 0.1, 1.0]  # Paramètre gamma pour le noyau RBF
}

# Initialisation du SVM
svm_clf = SVC(class_weight="balanced", random_state=42)

# Initialisation du GridSearchCV avec validation croisée (par exemple, 5 folds)
grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entraîner le modèle sur les données d'entraînement
grid_search.fit(X_tfidf_train_reduced, y_train_category)

# Afficher les meilleurs paramètres et la meilleure précision
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure précision obtenue : ", grid_search.best_score_)

# Utiliser le meilleur modèle pour faire des prédictions sur le test
best_svm_clf = grid_search.best_estimator_
y_pred_category = best_svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy sur le test :", accuracy_score(y_test_category, y_pred_category))
print("Classification Report sur le test :\n", classification_report(y_test_category, y_pred_category))

Apres 32 minutes d'execution , on a trouvé que les meilleures parametres pour SVM pour les catégories threat / identity_hate sont: C: 0.1, gamma: auto, kernel: rbf

In [ ]:
# Catégorie cible
category = "identity_hate"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Définir la grille de paramètres à tester
param_grid = {
    'C': [0.1, 1.0, 10.0],  # Valeurs de régularisation
    'kernel': ['linear', 'rbf'],  # Noyaux à tester
    'gamma': ['scale', 'auto', 0.1, 1.0]  # Paramètre gamma pour le noyau RBF
}

# Initialisation du SVM
svm_clf = SVC(class_weight="balanced", random_state=42)

# Initialisation du GridSearchCV avec validation croisée (par exemple, 5 folds)
grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entraîner le modèle sur les données d'entraînement
grid_search.fit(X_tfidf_train_reduced, y_train_category)

# Afficher les meilleurs paramètres et la meilleure précision
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure précision obtenue : ", grid_search.best_score_)

# Utiliser le meilleur modèle pour faire des prédictions sur le test
best_svm_clf = grid_search.best_estimator_
y_pred_category = best_svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy sur le test :", accuracy_score(y_test_category, y_pred_category))
print("Classification Report sur le test :\n", classification_report(y_test_category, y_pred_category))


Pour information : le temps d'execution de ce code est environ 20 minutes

In [ ]:
# Catégorie cible
category = "severe_toxic"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Définir la grille de paramètres à tester
param_grid = {
    'C': [0.1, 1.0, 10.0],  # Valeurs de régularisation
    'kernel': ['linear', 'rbf'],  # Noyaux à tester
    'gamma': ['scale', 'auto', 0.1, 1.0]  # Paramètre gamma pour le noyau RBF
}

# Initialisation du SVM
svm_clf = SVC(class_weight="balanced", random_state=42)

# Initialisation du GridSearchCV avec validation croisée (par exemple, 5 folds)
grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entraîner le modèle sur les données d'entraînement
grid_search.fit(X_tfidf_train_reduced, y_train_category)

# Afficher les meilleurs paramètres et la meilleure précision
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure précision obtenue : ", grid_search.best_score_)

# Utiliser le meilleur modèle pour faire des prédictions sur le test
best_svm_clf = grid_search.best_estimator_
y_pred_category = best_svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy sur le test :", accuracy_score(y_test_category, y_pred_category))
print("Classification Report sur le test :\n", classification_report(y_test_category, y_pred_category))

Les résultats pour la catégorie "severe_toxic" montrent une accuracy élevée de 0.9789, avec des performances notables dans la détection des cas positifs. Le F1-score pour la classe positive (1) est de 0.27, tandis que le rappel atteint 0.67, indiquant que le modèle identifie 67% des cas "severe_toxic". La précision pour cette classe est de 0.17, ce qui signifie que le modèle fait quelques faux positifs tout en détectant efficacement les vrais positifs. Le macro average F1-score de 0.63 reflète une bonne performance globale. Ces résultats démontrent une sensibilité accrue aux cas minoritaires, bien qu'il reste des opportunités d'amélioration. Pour optimiser davantage les performances du modèle, il serait bénéfique d'explorer des techniques supplémentaires pour traiter le déséquilibre des classes.

In [ ]:
# Catégorie cible
category = "severe_toxic"

# Labels pour la catégorie cible
y_train_category = train[category]
y_test_category = test[category]

# Initialisation et entraînement du modèle SVM: 
svm_clf = SVC(kernel="rbf", C=1.0,gamma="scale", class_weight="balanced",random_state=42)
svm_clf.fit(X_tfidf_train_reduced, y_train_category)

# Prédiction sur le jeu de test
y_pred_category = svm_clf.predict(X_tfidf_test_reduced)

# Évaluation des performances
print("Accuracy:", accuracy_score(y_test_category, y_pred_category))
print("Classification Report:\n", classification_report(y_test_category, y_pred_category))

Les résultats pour les catégories "threat" et "identity_hate" révèlent des performances très contrastées : Pour "threat", l'accuracy est très élevée (0.9966), mais le modèle échoue complètement à identifier les cas positifs. Les scores de précision, rappel et F1-score pour la classe positive sont tous à 0, indiquant que le modèle classe systématiquement tous les exemples comme négatifs. Cela suggère un problème sévère de déséquilibre des classes, avec seulement 34 exemples positifs sur 10000. Pour "identity_hate", les résultats sont inhabituels. L'accuracy est extrêmement basse (0.0125), mais le rappel pour la classe positive est de 1.00, signifiant que le modèle identifie tous les cas positifs. Cependant, la précision très faible (0.01) indique que le modèle classe presque tous les exemples comme positifs, résultant en de nombreux faux positifs. Ce comportement pourrait être dû à une sur-correction du déséquilibre des classes.

In [ ]:
categories = ["threat","identity_hate"]

# Résultats pour chaque catégorie
for category in categories:
    print(f"=== Catégorie : {category} ===")
    
    # Labels pour la catégorie cible
    y_train_category = train[category]
    y_test_category = test[category]
    
    # Initialisation et entraînement du modèle SVM
    svm_clf = SVC(kernel="rbf", C=0.1, gamma="auto", class_weight="balanced",random_state=42)
    svm_clf.fit(X_tfidf_train_reduced, y_train_category)
    
    # Prédiction sur le jeu de test
    y_pred_category = svm_clf.predict(X_tfidf_test_reduced)
    
    # Évaluation des performances
    print("Accuracy:", accuracy_score(y_test_category, y_pred_category))
    print("Classification Report:\n", classification_report(y_test_category, y_pred_category))

## ***Random Forest***
La méthode de Random Forest se base sur **plusieurs** *arbres de décisions* indépendants afin de prédire un modèle plus précis que ceux obtenu par chaque arbre individuellement.
Un arbre de décision est un ensemble d'algorithmes permettant de séparer au mieux nos données selon un certains nombre de décisions, représentées par des *branches*.
Un arbre est très sensible aux variations des données d'apprentissage.  C'est pour cela qu'une forêt est généralement privilégiée : en combinant les résultats de plusieurs arbres de décisions réalisés sur des données d'apprentissage variables, la forêt aléatoire réduit le risque d'erreurs dû à des changements dans les dites données. C'est donc une méthode robuste.\
De plus, à l'aide de cette méthode, nous pouvons observer l'importance de chaque variable grâce au score de Gini. C'est également une méthode très compréhensible pour un public non formé aux statistiques car elle est très visuelle. \

On peut alors se poser plusieurs question, comme :
- Combien de branches utiliser pour chaque arbre ? 
- Combien d'arbres en tout ?

In [ ]:
# Commentaires et cibles
X_train = train["comment_clean"]
Y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

X_test = test["comment_clean"]
Y_test = test_labels[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [ ]:
# Vectoriser
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.fit_transform(X_test)

In [ ]:
# Réduction de dimensions
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.fit_transform(X_test_tfidf)

Testons d'abord un modèle avec 10 arbres :

In [ ]:
# Forêt à 10 arbres
rf_10 = RandomForestClassifier(n_estimators = 10, random_state = 42)
multi_rf_10 = MultiOutputClassifier(rf_10)
# Permet de faire un modèle multi-label, i.e. de prédire plusieurs colonnes à la fois.

# Entrainement
multi_rf_10.fit(X_train_svd, Y_train)

# Prédiction
Y_pred = multi_rf_10.predict(X_test_svd)

# 135 secondes pour n_estimators = 10.

Recherchons maintenant les meilleurs valeurs pour les hyperparamètres suivant :
- `n_estimators` : nombre d'arbres.
- `min_samples_split` : nombre minimal d'échantillons pour diviser un noeud.
- `min_samples_leaf` : nombre minimal d'échantillons par feuille.
- `max_depth` : profondeur maximale des arbres.

In [ ]:
# Une première recherche a permis de trouver la valeur optimale pour les 3 premiers paramètre en 2264 secondes.
# On les fixe donc aux valeurs de sortie (10, 6, 1 resp) et on se concentre ici sur max_depth.
rf = RandomForestClassifier(min_samples_leaf = 1, min_samples_split = 6, n_estimators = 10, random_state = 42)
multi_rf = MultiOutputClassifier(rf)

# Grille des hyperparamètres
param_grid = {
    # "estimator__n_estimators": [5, 10, 20],          # Nombre d'arbres
    # "estimator__min_samples_split": [4, 6, 8],       # Nombre minimal d'échantillons pour diviser un noeud
    # "estimator__min_samples_leaf": [1, 2, 3],        # Nombre minimal d'échantillons par feuille
    "estimator__max_depth": [5, 10, 20, 50, 100]       # Profondeur maximale des arbres
}
# Les noms des hyperparamètres sont précédés de "estimator__" car le modèle est encapsulé dans 'multi_rf'.

# GridSearchCV
grid_search = GridSearchCV(multi_rf, param_grid, cv = 3, scoring = "f1_micro", verbose = 1, n_jobs = -1)
# cv : divise le dataset en i et entraîne/teste sur ces différentes combinaisons.
# scoring : métrique utilisée pour évaluer le modèle. Ici, "f1_micro" est utilisé car multi-label (f1 = métrique de base).
# verbose : quantité de détails affichés lors de l'exécution.
# n_jobs = -1 : tous les cv sont exécuté en parallèle pour accélérer la recherche.

grid_search.fit(X_train_svd, Y_train)

# Meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)

# Meilleure max_depth = 50 (trouvé en 230 secondes).

il aura donc fallu en tout environ 40 minutes pour trouver nos 4 hyperparamètres.

In [ ]:
# Prédictions avec le meilleur modèle de random forest multi-label
rf_opt = RandomForestClassifier(min_samples_leaf = 1, 
                                min_samples_split = 6, 
                                n_estimators = 10, 
                                max_depth = 50,
                                random_state = 42)
multi_rf_opt = MultiOutputClassifier(rf_opt)

multi_rf_opt.fit(X_train_svd, Y_train)

Y_pred = multi_rf_opt.predict(X_test_svd)

Il faut environ 4 minutes pour faire l'entraînement/prédiction.

Intéressons-nous enfin à l'analyse des résultats prédit par la Random Forest.

In [ ]:
print("Rapport pour chaque catégorie : \n")
for i, column in enumerate(Y_train.columns):
    print(pd.DataFrame(confusion_matrix(Y_test.iloc[:, i], Y_pred[:, i]),
             index = ["0 donné", "1 donné"],
             columns = ["0 prédit", "1 prédit"]))

In [ ]:
# Initialiser un DataFrame pour stocker les résultats
results = []

for i, column in enumerate(Y_train.columns):
    # Calculer le rapport de classification
    report = classification_report(Y_test.iloc[:, i], Y_pred[:, i], 
                          zero_division = 0,  # Évite les avertissements
                          output_dict = True
                         )
    
    # Calculer l'accuracy pour chaque étiquette
    accuracy = accuracy_score(Y_test.iloc[:, i], Y_pred[:, i])
    
    # Ajouter les résultats au tableau
    results.append({
        "Label": column,
        "Precision": report['weighted avg']['precision'],
        "Recall": report['weighted avg']['recall'],
        "F1-Score": report['weighted avg']['f1-score'],
        "Accuracy": accuracy  # Ajouter l'accuracy à chaque étiquette
    })

# Afficher les résultats sous forme de tableau
results_df = pd.DataFrame(results)
print(results_df)
print("\n \n Précision générale : ", accuracy_score(Y_test, Y_pred), "\n")

Ce modèle permet de conduire à un taux d’erreur d’environ 10% concernant les prédictions, cela reste plus que raisonnable. Malgré tout, le temps d'exécution du modèle, en comptant la recherche d'optimisation d'hyper-paramètres, reste assez long, ce qui peut poser problème en terme de coût computationnnel suivant la machine. Un modèle plus rapide pourrait donc être à privilégier afin de réduire ce coût.

# Méthodes des K-Nearest Neighbors

La méthode des K-Nearest Neighbors est une méthode de classification non-supervisée et non-paramétrique, ce qui signfique qu'il n'est pas nécéssaire de faire des hypothèses concernant la distribution des données. Les méthodes non-paramétriques sont robustes et flexibles, ce qui induit qu'elles peuvent traiter des données biaisées, présentant des valeurs abbérantes ou ayant des échelles ou des unités différentes. Cette méthode se base sur le calcul des distances entre les différents individus. Lors de l'implémentation de la méthode des KNN, il est possible de choisir la distance calculée : distance euclidienne, distance de Ward, distance de Manhattan, etc. Cette distance peut être chosie par validation croisée. Dans notre cas, nous avons gardé la distance par defaut qui est la distance euclidienne car, après simulation, c'était celle avec le taux d'erreur le plus faible. \
Cette méthode consiste à regrouper les individus de manière itérative, en minimisant la disance avec leurs voisins selon k clusters. Il est également possible d'optimiser le paramètre k par validation croisée, ce qui a été fait dans ce cas présent. 

In [ ]:
# Importation des packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

# Initialisation
X_train = train[:, "comment_clean"]
Y_train = train[:, "toxic"]
X_test = test[:, "comment_clean"]
Y_test = test[:, "toxic"]

# Choix de la valeur k par validation croisée
tx_erreur = []
k = range(1, 100)

for i in k:
    models = KNeighborsClassifier(n_neighbors=i)
    models.fit(X_train, Y_train)
    pred_i = models.predict(X_test)
    tx_erreur.append(np.mean(pred_i != Y_test))

best_k = []
for i in range (0, 99):
    if tx_erreur[i+1] <= tx_erreur[i]
    best_k = i+1

print(best_k)

# Algorithme de création du modèle K-NN
model = KNeighborsClassifier(n_neighbors=best_k)
model.fit(X_train, Y_train)

# Prédictions des invidus test
prediction = model.predict(X_test)

# Calcul du taux d'erreur
print(confusion_matrix(Y_test, prediction))
print(classification_report(Y_test, predictions))

# tx_erreur = mean((prediction - data[, "toxic"])^2)

# Affichage des résultats
print(tx_erreur)

# ***Comparaison des modèles***

## ***SVM vs Régression logistique***
En raison des limitations matérielles, j’ai utilisé une sous-partie du dataset (15 000 observations pour l’entraînement et 10 000 pour les tests), contrairement à mes collègues qui ont travaillé sur l’ensemble des données. Cela rend mes résultats avec le SVM difficilement comparables, car le modèle n’a pas été exposé à l’intégralité des données, ce qui pourrait biaiser les métriques obtenues, comme des scores plus élevés sur cet échantillon restreint. Le SVM présente des avantages notables : il est particulièrement performant sur des datasets de taille modérée et des classes déséquilibrées, grâce à l’argument class_weight="balanced", et il modélise des relations complexes via des noyaux comme RBF tout en résistant au surapprentissage. De plus, l’optimisation des hyperparamètres (par GridSearch) a permis d’améliorer ses performances. Cependant, son principal inconvénient réside dans sa scalabilité : son temps d’exécution devient prohibitif sur des datasets volumineux ou très dimensionnels. Pour remédier à cela, il est souvent nécessaire de réduire à la fois le nombre de points (échantillonnage) et les dimensions (via PCA), car réduire uniquement les dimensions ne suffit pas à diminuer significativement le temps d’exécution. En comparaison, la régression logistique est plus rapide, même avec de grands volumes de données, grâce à sa simplicité basée sur des calculs matriciels, ce qui la rend plus adaptée aux très grands datasets tout en offrant des performances compétitives. En conclusion, bien que le SVM soit un excellent choix pour des problèmes complexes ou des datasets modérés, une comparaison équitable avec d’autres modèles, comme la régression logistique, nécessiterait d’évaluer tous les modèles sur l’ensemble des données pour tirer des conclusions définitives.

## ***Régression logistique vs Random Forest***

Au vu de nos résultats de prédiction, les 2 modèles sont aussi précis l'un que l'autre pour prédire les étiquettes des commentaires. Egalement, les deux méthodes peuvent permettre de regarder l'importance des variables avec le score de permutation pour la régression logistique et le score de Gini pour la forêt aléatoire.\
Toutefois, le temps d'exécution de la régression logistique étant bien inférieure à celui de la Random Forest, le premier est donc ici à privilégier.
De plus, la régression logistique permet d'effectuer des tests a postériori pour vérifier la significativité et d'obtenir plus d'indicateurs tels que les odds-ratio. 


## ***Conclusion***
Des trois modèles étudiés précédemment, la régression logistique donc semble le mieux adapté pour nos données.

# ***Modèle implémenter en CLI***

## Use your model

**Use the best model to build a Command-Line Interface (*CLI*) that is launched by the command `./cli.py [options]` using the `argsparse` module, and that accepts in stdin (standard input) english sentences and classifies them, displaying the result and interesting metrics if relevant.**

In [ ]:
import pickle as pk
import argparse
import sys